In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
def process_chunk(chunk):
        print(f'Processing chunk with {len(chunk)} rows')
        display(chunk)


with pd.read_json('data/quotes-2015.json.bz2', lines=True, compression='bz2', chunksize=10000) as df_reader:
    for chunk in df_reader:
        process_chunk(chunk)
        break

Processing chunk with 10000 rows


,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2015-08-31-000271,... a great day for veterans here in Littleton...,Jeanne Shaheen,[Q270316],2015-08-31 02:10:00,2,"[[Jeanne Shaheen, 0.742], [None, 0.2359], [Kel...",[http://www.unionleader.com/article/20150831/N...,E
1,2015-12-08-029916,How FFA scored 32 own goals in 18 months and C...,None,[],2015-12-08 00:00:00,2,"[[None, 0.563], [David Gallop, 0.437]]",[http://feeds.theroar.com.au/~r/theroar/~3/tZ3...,E
2,2015-09-10-000206,[ Amy ] was placed under an unacceptable amoun...,None,[],2015-09-10 21:18:59,1,"[[None, 0.9634], [Amy Robinson, 0.0366]]",[http://www.thefashionspot.com/buzz-news/lates...,E
3,2015-07-23-032707,How High Will These Numbers Go?,None,[],2015-07-23 14:57:49,3,"[[None, 0.9019], [Chubby Checker, 0.0981]]","[http://www.billboard.com/node/6641719, http:/...",E
4,2015-10-04-000115,[ Ban ] recalls that hospitals and medical per...,Ban Ki-moon,[Q1253],2015-10-04 08:27:38,2,"[[Ban Ki-moon, 0.8399], [None, 0.1601]]",[http://muslimnews.co.uk/news/middle-east/afgh...,E
...,...,...,...,...,...,...,...,...,...
9995,2015-04-27-046064,The majority of Preterm's abortion customers a...,None,[],2015-04-27 16:54:00,1,"[[None, 0.9144], [George Tiller, 0.0856]]",[http://www.lifenews.com/2015/04/27/watch-abor...,E
9996,2015-06-23-031044,"Len and I have been very involved and careful,",Sandy Lerner,[Q7549],2015-06-23 02:21:05,1,"[[Sandy Lerner, 0.9496], [None, 0.0504]]",[http://washingtonian.com/articles/people/lern...,E
9997,2015-03-10-074126,The majority of the drivers in last year's eve...,Chris Blair,[Q1077008],2015-03-10 21:51:36,1,"[[Chris Blair, 0.889], [None, 0.111]]",[http://paddocktalk.com/news/html/modules.php?...,E
9998,2015-04-15-053845,less than where we thought when we started the...,Stacy Smith,[Q7596057],2015-04-15 09:44:19,1,"[[Stacy Smith, 0.9769], [None, 0.0231]]",[http://iol.co.za:80/business/companies/intel-...,E
